# Second stage of the OSIOSN project

### Creating and training baseline model

In [1]:
import os
import numpy as np
import torch
from torch import nn, cat
from torch.nn import Conv1d, ConvTranspose1d, MaxPool1d, Dropout, ReLU, Sequential, Sigmoid
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [2]:
DATASET_PATH = "E:\\ml-data\\masters-thesis\\physionet.org\\files\\apnea-ecg\\1.0.0\\dataset"

In [3]:
x_train = np.expand_dims(np.load(os.path.join(DATASET_PATH, "x_train.npy")), 1)[:10000]
y_train = np.expand_dims(np.load(os.path.join(DATASET_PATH, "y_train.npy")), 1)[:10000]

x_test = np.expand_dims(np.load(os.path.join(DATASET_PATH, "x_test.npy")), 1)[:2500]
y_test = np.expand_dims(np.load(os.path.join(DATASET_PATH, "y_test.npy")), 1)[:2500]

x_val = np.expand_dims(np.load(os.path.join(DATASET_PATH, "x_val.npy")), 1)[:1000]
y_val = np.expand_dims(np.load(os.path.join(DATASET_PATH, "y_val.npy")), 1)[:1000]

In [4]:
num_epochs = 20
batch_size = 20
lr = 1e-3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Training will be performed with:',device)

Training will be performed with: cuda:0


In [5]:
# data shape [batch_size, channels, samples]

class Conv1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size):
    super(Conv1DBlock, self).__init__()
    self.conv1d = Conv1d(in_channels, out_channels, kernel_size, padding="same")
    self.relu = ReLU(inplace=True)

  def forward(self, x):
    x = self.conv1d(x)
    x = self.relu(x)
    return x

class DoubleConv1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size):
    super(DoubleConv1DBlock, self).__init__()
    self.double_conv = Sequential(
      Conv1DBlock(in_channels, out_channels, kernel_size),
      Conv1DBlock(out_channels, out_channels, kernel_size)
    )

  def forward(self, x):
    return self.double_conv(x)


class DownSample1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, dropout=0.3):
    super(DownSample1DBlock, self).__init__()
    self.double_conv = DoubleConv1DBlock(in_channels, out_channels, kernel_size)
    self.maxpool_1d = MaxPool1d(kernel_size=2)
    self.dropout = Dropout(dropout)

  def forward(self, x):
    x = self.double_conv(x)
    pool = self.maxpool_1d(x)
    return x, self.dropout(pool)


class UpSample1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, dropout=0.3):
    super(UpSample1DBlock, self).__init__()
    self.conv1d_transpose = ConvTranspose1d(in_channels, out_channels, kernel_size=2, stride=2)
    self.double_conv = DoubleConv1DBlock(in_channels, out_channels, kernel_size)
    self.dropout = Dropout(dropout)

  def forward(self, x, conv_features):
    x = self.conv1d_transpose(x)
    x = cat([x, conv_features], dim=1)
    x = self.dropout(x)
    return self.double_conv(x)

In [6]:
class ECGDataset(Dataset):
  def __init__(self, data, labels):
    super(ECGDataset, self).__init__()
    self.data = data
    self.labels = labels

  def __len__(self):
    return self.data.shape[0]
  
  def __getitem__(self, idx):
    sample = torch.tensor(self.data[idx], dtype=torch.float32)
    label = torch.tensor(self.labels[idx], dtype=torch.float32)
    return sample, label

In [7]:
class RPNet(nn.Module):
    def __init__(self, in_channels, out_channels, lr=1e-3):
        super(RPNet, self).__init__()
        self.d1 = DownSample1DBlock(in_channels, 16, 3)
        self.d2 = DownSample1DBlock(16, 32, 3)
        self.d3 = DownSample1DBlock(32, 64, 3)

        self.bottleneck = DoubleConv1DBlock(64, 128, 3)

        self.u1 = UpSample1DBlock(128, 64, 3)
        self.u2 = UpSample1DBlock(64, 32, 3)
        self.u3 = UpSample1DBlock(32, 16, 3)

        self.output = Conv1d(16, out_channels, kernel_size=1)
        self.sigmoid = Sigmoid()

        self.criterion = nn.BCELoss()
        self.optimizer = Adam(self.parameters(), lr=lr)


    def forward(self, x):
        # x = x.transpose(1,2)

        f1, p1 = self.d1(x)
        f2, p2 = self.d2(p1)
        f3, p3 = self.d3(p2)

        bottleneck = self.bottleneck(p3)

        u1 = self.u1(bottleneck, f3)
        u2 = self.u2(u1, f2)
        u3 = self.u3(u2, f1)

        output = self.output(u3)
        return self.sigmoid(output)

    def train_model(self, x_train, y_train, epochs=10, batch_size=1, x_val=None, y_val=None,):
        self.batch_size = batch_size
        dataset = ECGDataset(x_train, y_train)
        train_loader = DataLoader(dataset, batch_size, shuffle=False)

        if x_val is not None:
            validation_dataset = ECGDataset(x_val, y_val)
            validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

        for epoch in range(epochs):
            running_loss = 0.0
            num_r_peaks = 0.0
            num_correct = 0.0

            all_outputs = []
            all_labels = []

            self.train()
            for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
                x, y = x.to(device), y.to(device)

                self.optimizer.zero_grad()
                outputs = self(x)

                loss = self.criterion(outputs, y)
                running_loss += loss.item()

                loss.backward()
                self.optimizer.step()

                outputs = outputs.cpu().detach().numpy()
                y = y.cpu().detach().numpy()

                num_r_peaks += np.where(y == 1)[0].shape[0]
                num_correct += np.where((outputs > 0.5) & (y == 1))[0].shape[0]

                all_outputs.extend(outputs.flatten())
                all_labels.extend(y.flatten())
            
            all_outputs = np.array(all_outputs)
            all_labels = np.array(all_labels)
            y_pred_binary = (all_outputs > 0.5).astype(int)

            print(f"====Epoch [{epoch + 1}/{epochs}]====")
            print(f"\nTrain Loss: {running_loss / len(train_loader):.4f}")
            self.calculate_metrics(num_correct, num_r_peaks, all_labels, y_pred_binary, phase="Train")
        
            if x_val is not None:
                self.validate(validation_loader)
  
    def validate(self, validation_loader):
        self.eval()
        running_vloss = 0.0
        num_r_peaks = 0.0
        num_correct = 0.0

        all_outputs = []
        all_labels = []

        with torch.no_grad():
            for i, (x_val, y_val) in tqdm(enumerate(validation_loader), total=len(validation_loader)):
                x, y = x_val.to(device), y_val.to(device)
                outputs = self(x)

                loss = self.criterion(outputs, y)
                running_vloss += loss.item()

                outputs = outputs.cpu().detach().numpy()
                y = y.cpu().detach().numpy()

                num_r_peaks += np.where(y == 1)[0].shape[0]
                num_correct += np.where((outputs > 0.5) & (y == 1))[0].shape[0]

                all_outputs.extend(outputs.flatten())
                all_labels.extend(y.flatten())

            all_outputs = np.array(all_outputs)
            all_labels = np.array(all_labels)
            y_pred_binary = (all_outputs > 0.5).astype(int)

            print(f"\nValidation Loss: {running_vloss / len(validation_loader):.4f}")
            self.calculate_metrics(num_correct, num_r_peaks, all_labels, y_pred_binary, phase="Validation")
    
    def test_model(self, x_test, y_test, plot=False):
        test_dataset = ECGDataset(x_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

        
        running_loss = 0.0
        num_r_peaks = 0.0
        num_correct = 0.0

        all_outputs = []
        all_labels = []
        
        self.eval()
        with torch.no_grad():
            for i, (x_test, y_test) in tqdm(enumerate(test_loader), total=len(test_loader)):
                x, y = x_test.to(device), y_test.to(device)
                outputs = self(x)

                loss = self.criterion(outputs, y)
                running_loss += loss.item()

                outputs = outputs.cpu().detach().numpy()
                y = y.cpu().detach().numpy()

                num_r_peaks += np.where(y == 1)[0].shape[0]
                num_correct += np.where((outputs > 0.5) & (y == 1))[0].shape[0]

                all_outputs.extend(outputs.flatten())
                all_labels.extend(y.flatten())

                if plot and (i % (len(test_loader) / 10) == 0):
                    ecg = x[0].cpu().detach().numpy()
                    gt = y[0]
                    pred = outputs[0]

                    plt.figure()
                    plt.plot(ecg)
                    plt.plot(gt)
                    plt.plot(pred)
                    plt.legend(["ECG", "Ground Truth", "Prediction"])
                    plt.grid()
                    plt.show()
                
        all_outputs = np.array(all_outputs)
        all_labels = np.array(all_labels)
        y_pred_binary = (all_outputs > 0.5).astype(int)

        print(f"\nTest Loss: {running_loss / len(test_loader):.4f}")
        self.calculate_metrics(num_correct, num_r_peaks, all_labels, y_pred_binary, phase="Test")
    
    # we only care about the precision of the R_peaks (binary class 1) and we about the false positive rate
    def calculate_metrics(self, num_correct_peaks, total_peaks, y_true, y_pred_binary, phase="Train"):
        accuracy = num_correct_peaks / total_peaks * 100

        f1 = f1_score(y_true, y_pred_binary)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

        print(f"{phase} Accuracy: {accuracy:.5f} %")
        print(f"{phase} F1 Score: {f1:.5f}")
        print(f"{phase} TPR: {tpr:.5f}")
        print(f"{phase} FPR: {fpr:.5f}\n")

In [8]:
model = RPNet(in_channels=1, out_channels=1)
model.to(device)

RPNet(
  (d1): DownSample1DBlock(
    (double_conv): DoubleConv1DBlock(
      (double_conv): Sequential(
        (0): Conv1DBlock(
          (conv1d): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=same)
          (relu): ReLU(inplace=True)
        )
        (1): Conv1DBlock(
          (conv1d): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=same)
          (relu): ReLU(inplace=True)
        )
      )
    )
    (maxpool_1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (d2): DownSample1DBlock(
    (double_conv): DoubleConv1DBlock(
      (double_conv): Sequential(
        (0): Conv1DBlock(
          (conv1d): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=same)
          (relu): ReLU(inplace=True)
        )
        (1): Conv1DBlock(
          (conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=same)
          (relu): ReLU(inplace=True)
        )
      )
    )
    (maxpool_1d)

In [9]:
model.train_model(x_train=x_train, y_train=y_train, epochs=10, x_val=x_val, y_val=y_val)

100%|██████████| 10000/10000 [01:41<00:00, 98.19it/s]


====Epoch [1/10]====

Train Loss: 0.0163
Train Accuracy: 66.32433 %
Train F1 Score: 0.71314
Train TPR: 0.66324
Train FPR: 0.00220



100%|██████████| 1000/1000 [00:03<00:00, 281.46it/s]



Validation Loss: 0.0108
Validation Accuracy: 77.32684 %
Validation F1 Score: 0.79503
Validation TPR: 0.77327
Validation FPR: 0.00193



100%|██████████| 10000/10000 [01:48<00:00, 91.87it/s]


...


Validation Loss: 0.0077
Validation Accuracy: 85.60606 %
Validation F1 Score: 0.86405
Validation TPR: 0.85606
Validation FPR: 0.00141



100%|██████████| 10000/10000 [01:35<00:00, 104.75it/s]


====Epoch [10/10]====

Train Loss: 0.0090
Train Accuracy: 82.45830 %
Train F1 Score: 0.83461
Train TPR: 0.82458
Train FPR: 0.00169



100%|██████████| 1000/1000 [00:03<00:00, 303.92it/s]



Validation Loss: 0.0078
Validation Accuracy: 85.14610 %
Validation F1 Score: 0.86113
Validation TPR: 0.85146
Validation FPR: 0.00141

